Funcion objetivo:

Maximizar z = 70x1 + 91x2 + 50x2 + 61x4 + 21x5

Minimizar z = 170x1 + 310x2 + 60x3 + 101x4 + 11x5

sujeto a:

      x1 ≤ 15
      x2 ≤ 10
      x3 ≤ 25
      x4 ≤ 4
      x5 ≤ 30
      
      170x1 + 310x2 ≤ 3800
      60x3 + 101x4 ≤ 2800
      101x4 + 11x5 ≤ 3500

In [1]:
import random
import numpy as np

Nodo y Arco Consistencia


In [2]:
# Nodo Consistencia
domains = {
    'x1': [i for i in range(0,16)],
    'x2': [i for i in range(0,11)],
    'x3': [i for i in range(0,26)],
    'x4': [i for i in range(0,5)],
    'x5': [i for i in range(0,31)],
}

In [3]:
# Restricciones
constraints = {

    ('x1', 'x2'): lambda x1, x2:  170*x1 <= 3800-310*x2,
    ('x2', 'x1'): lambda x2, x1:  3800-310*x2 >= 170*x1,

    ('x3', 'x4'): lambda x3, x4:  60*x3 <= 2800-101*x4,
    ('x4', 'x3'): lambda x4, x3:  2800-101*x4 >= 60*x3,

    ('x3', 'x5'): lambda x3, x5:  60*x3 <= 3500-11*x5,
    ('x5', 'x3'): lambda x5, x3:  3500-11*x5 >= 60*x3,

}

In [4]:
# Implementación de AC3
def revise(x, y):
    revised = False
    x_domain = domains[x]
    y_domain = domains[y]
    all_constraints = [
        constraint for constraint in constraints if constraint[0] == x and constraint[1] == y]
    for x_value in x_domain:
        satisfies = False
        for y_value in y_domain:
            for constraint in all_constraints:
                constraint_func = constraints[constraint]
                if constraint_func(x_value, y_value):
                    satisfies = True
        if not satisfies:
            x_domain.remove(x_value)
            revised = True
    return revised

def ac3(arcs):
    queue = arcs[:]
    while queue:
        (x, y) = queue.pop(0)
        revised = revise(x, y)
        if revised:
            neighbors = [neighbor for neighbor in arcs if neighbor[1] == x]
            queue = queue + neighbors


In [5]:
# Arcos de las restricciones del problema
arcs = [
    ('x1','x2'), ('x2','x1'),
    ('x3','x4'), ('x4','x3'),
    ('x3', 'x5'), ('x5','x3')
]

ac3(arcs)

print("x1: " + str(domains['x1']))
print("x2: " + str(domains['x2']))
print("x3: " + str(domains['x3']))
print("x4: " + str(domains['x4']))
print("x5: " + str(domains['x5']))

x1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
x2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
x3: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
x4: [0, 1, 2, 3, 4]
x5: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [6]:
class Problema:
  def __init__(self):
    self.dimensiones = 5
    self.limites = {
      'x1': (0, 15),
      'x2': (0, 10),
      'x3': (0, 25),
      'x4': (0, 4),
      'x5': (0, 30)
    }

    self.costos = {
      'x1': 170,
      'x2': 310,
      'x3': 60,
      'x4': 101,
      'x5': 11
    }

    self.presMaxTv = 3800
    self.presMaxDyR = 2800
    self.presMaxDyR = 3500

  def check(self, x):
    # Chequeo de restricciones de presupuesto
    costo_tv = self.costos['x1'] * x[0] + self.costos['x2'] * x[1]
    costo_print = self.costos['x3'] * x[2] + self.costos['x4'] * x[3]
    costo_combined = self.costos['x3'] * x[2] + self.costos['x5'] * x[4]
        
    if costo_tv > self.presMaxTv:
      return False
    if costo_print > self.presMaxDyR:
      return False
    if costo_combined > self.presMaxDyR:
      return False

    # Chequeo de límites de cantidad de anuncios
    for i, (c_min, c_max) in enumerate(self.limites.values()):
      if not (c_min <= x[i] <= c_max):
        return False

    return True

  def eval(self, x):
    # Se evalua el fitness uti
        return 70 * x[0] + 91 * x[1] + 50 * x[2] + 61 * x[3] + 21 * x[4]
  
  def sigmoideDim(self, x):
    a = 1 / (1 + np.exp(-8*x + 4))
    if a > 0.5:
        a = 1
    else:
        a = 0
    return a

  def sigmoideP(self, arr):
    result = []
    for n in arr:
      aux = n - int(n)
      s = self.sigmoideDim(aux)
      n = int(n) + s
      result.append(n)
    return np.array(result)


In [7]:
class Particula:
  def __init__(self, problema):
    self.problema = problema
    self.x = np.zeros(problema.dimensiones)
    self.inicializacion()


  def inicializacion(self):
    for j in range(self.problema.dimensiones):
      c_min, c_max = list(self.problema.limites.values())[j]
      self.x[j] = (c_min + random.random() * (c_max - c_min))

  def esFactible(self, x):
    # Codigo
    return self.problema.check(self.x)

  def esMejorQue(self, comp):
    # Codigo
    return self.fit() > comp.fit()

  def fit(self):
    # Codigo
    return self.problema.eval(self.x)

  def __str__(self):
    # Representación en cadena de la partícula
    return f"fit:{self.fit()} x:{self.x}"


In [8]:
class EquilibriumOptimizer1:
    def __init__(self, problema, n, MAX_ITER, a1, a2, GP):
        self.problema = problema
        self.nParticulas = n
        self.maxIter = MAX_ITER
        self.a1 = a1
        self.a2 = a2
        self.GP = GP
        self.V = 1
        self.enjambre = []
        self.eq_candidatos = [Particula(problema) for _ in range(n)]

        self.lower_band = [ self.problema.limites[f"x{i+1}"][0] for i in range( len( self.problema.limites.keys() ) ) ]
        self.upper_band = [ self.problema.limites[f"x{i+1}"][-1] for i in range( len( self.problema.limites.keys() ) ) ]


    def inicializarPoblacion(self):
        for _ in range(self.nParticulas):
            while True:
                particula = Particula(self.problema)
                if particula.esFactible(particula.x):
                    self.enjambre.append(particula)
                    break
    
    def updateCandidatosEq(self):
        for particula in self.enjambre:

            if particula.esMejorQue(self.eq_candidatos[0]):
                self.eq_candidatos = [particula] + self.eq_candidatos[:-1]

            elif particula.esMejorQue(self.eq_candidatos[1]):
                self.eq_candidatos = [self.eq_candidatos[0], particula] + self.eq_candidatos[1:-1]

            elif particula.esMejorQue(self.eq_candidatos[2]):
                self.eq_candidatos = self.eq_candidatos[:2] + [particula] + self.eq_candidatos[2: -1]
            
            elif particula.esMejorQue(self.eq_candidatos[3]):
                self.eq_candidatos[3] = particula
                
    def construirEqPool(self):
        eqPromedio = Particula(self.problema)
        eqPromedio.x = np.mean([ candidato.x for candidato in self.eq_candidatos ], axis = 0).tolist()
        return self.eq_candidatos + [ eqPromedio ]
    
    def evolucion(self):

        for iter in range(1, self.maxIter + 1):

            print(f"Iteracion no: {iter}")
            
            self.updateCandidatosEq()
            eq_pool = self.construirEqPool()
            for particula in eq_pool:
                print(particula)

            # Calcular t segun Eq. (9)
            t = (1 - iter / self.maxIter) ** ( self.a2 * iter/self.maxIter )

            for particula in self.enjambre:

                eq_candidato = random.choice(eq_pool)                        # Eleccion randomica de un candidato del eq_pool
                vectorLambda = np.random.rand(self.problema.dimensiones)     # Valor randomico del 0 al 1 para la Eq. (11)
                vectorR = np.random.rand(self.problema.dimensiones)          # valor randomico del 0 al 1 para la Eq. (11)

                # Eq. (11)
                F = self.a1 * np.sign(vectorR - 0.5) * ( np.exp(-vectorLambda * t) - 1 )

                # Eq. (15)
                GCP = np.where( np.random.rand(self.problema.dimensiones) >= self.GP , 0.5 * random.random(), 0 )

                # Eq. (14)
                G0 = GCP * ( eq_candidato.x - vectorLambda * particula.x )

                # Eq. (13)
                G = G0 * F

                # Eq. (16)
                particula.x = eq_candidato.x + ( (particula.x - eq_candidato.x) * F )  + (G / vectorLambda) * (1 - F)
        
                # np.clip
                particula.x = np.clip(particula.x, self.lower_band, self.upper_band)

                particula.x = np.where(particula.x % 1 == 0, particula.x, np.round(particula.x))
                

                

    def solve(self):
        self.inicializarPoblacion()
        self.evolucion()
        self.updateCandidatosEq()
        mejoresParticulas = self.eq_candidatos
        print("Mejores particulas: ")
        for particula in mejoresParticulas:
            print(particula)


In [9]:
class EquilibriumOptimizer2:
    def __init__(self, problema, n, MAX_ITER, a1, a2, GP):
        self.problema = problema
        self.nParticulas = n
        self.maxIter = MAX_ITER
        self.a1 = a1
        self.a2 = a2
        self.GP = GP
        self.V = 1
        self.enjambre = []
        self.eq_candidatos = [Particula(problema) for _ in range(4)]

        self.lower_band = [ self.problema.limites[f"x{i+1}"][0] for i in range( len( self.problema.limites.keys() ) ) ]
        self.upper_band = [ self.problema.limites[f"x{i+1}"][-1] for i in range( len( self.problema.limites.keys() ) ) ]


    def inicializarPoblacion(self):
        for _ in range(self.nParticulas):
            while True:
                particula = Particula(self.problema)
                particula.x = self.problema.sigmoideP(particula.x)
                print(particula.x)
                if particula.esFactible(particula.x):
                    self.enjambre.append(particula)
                    break
    
    def updateCandidatosEq(self):
        for particula in self.enjambre:
            for i in range(len(self.eq_candidatos)):
                if particula.esMejorQue(self.eq_candidatos[i]):
                    self.eq_candidatos[i] = particula
                    break
                
    def construirEqPool(self):
        eqPromedio = Particula(self.problema)
        eqPromedio.x = np.mean([ candidato.x for candidato in self.eq_candidatos ], axis = 0).tolist()
        eqPromedio.x = self.problema.sigmoideP(eqPromedio.x)
        return self.eq_candidatos + [ eqPromedio ]
    
    def evolucion(self):

        for iter in range(1, self.maxIter + 1):

            print(f"\n Iteracion no: {iter}")
            
            self.updateCandidatosEq()
            eq_pool = self.construirEqPool()

            for i in range(5):
                if i <= 3:
                    print(f"Mejor particula ${i + 1}")
                    print(eq_pool[i])
                else:
                    print("Promedio: ")
                    print(eq_pool[i])

            # Calcular t segun Eq. (9)
            t = (1 - iter / self.maxIter) ** ( self.a2 * iter/self.maxIter )

            for particula in self.enjambre:

                while True:

                    eq_candidato = random.choice(eq_pool)                        # Eleccion randomica de un candidato del eq_pool
                    vectorLambda = np.random.rand(self.problema.dimensiones)     # Valor randomico del 0 al 1 para la Eq. (11)
                    vectorR = np.random.rand(self.problema.dimensiones)          # valor randomico del 0 al 1 para la Eq. (11)

                    # Eq. (11)
                    F = self.a1 * np.sign(vectorR - 0.5) * ( np.exp(-vectorLambda * t) - 1 )

                    # Eq. (15)
                    GCP = np.where( np.random.rand(self.problema.dimensiones) >= self.GP , 0.5 * random.random(), 0 )

                    # Eq. (14)
                    G0 = GCP * ( eq_candidato.x - vectorLambda * particula.x )

                    # Eq. (13)
                    G = G0 * F

                    # Eq. (16)
                    particula.x = eq_candidato.x + ( (particula.x - eq_candidato.x) * F )  + (G / vectorLambda) * (1 - F)

                    particula.x = self.problema.sigmoideP(particula.x)
                    
                    # np.clip
                    #particula.x = np.clip(particula.x, self.lower_band, self.upper_band)

                    if (particula.esFactible(particula.x)):
                        #print("particula era factible")
                        break
                

    def solve(self):
        self.inicializarPoblacion()
        self.evolucion()
        self.updateCandidatosEq()
        mejoresParticulas = self.eq_candidatos
        print("\n\nMejores particulas: ")
        for particula in mejoresParticulas:
            print(particula)

In [14]:
# Cantidad de particulas = 5
n = 10
# Numero maximo de iteraciones
MAX_ITER = 15
# Constantes de explotacion y explotacion
a1 = 2
a2 = 1
#
GP = 0.5

# Ejecutar el optimizador
problema = Problema()
optimizer = EquilibriumOptimizer2(problema, n, MAX_ITER, a1, a2, GP)
optimizer.solve()


[9 7 6 1 0]
[ 8  5 24  1 12]
[ 9  6  2  1 28]
[12  2  1  3  6]
[14  5 16  0 12]
[ 1  1  7  2 30]
[ 9  0 12  3  1]
[ 7  8  8  2 20]
[ 3  7 25  0 16]
[ 6  1 20  1 20]
[11  4  3  0 15]

 Iteracion no: 1
Mejor particula $1
fit:2528 x:[ 8  5 24  1 12]
Mejor particula $2
fit:2592.050909110547 x:[13.80077296  4.88058233  9.53070765  3.58672215 23.16849413]
Mejor particula $3
fit:2433 x:[ 3  7 25  0 16]
Mejor particula $4
fit:1992 x:[ 6  1 20  1 20]
Promedio: 
fit:2363 x:[ 8  4 20  1 18]

 Iteracion no: 2
Mejor particula $1
fit:2278 x:[ 3  7 24  0 11]
Mejor particula $2
fit:2592.050909110547 x:[13.80077296  4.88058233  9.53070765  3.58672215 23.16849413]
Mejor particula $3
fit:2196 x:[ 8  3 21  1 12]
Mejor particula $4
fit:2109 x:[13  5  9  0 14]
Promedio: 
fit:2261 x:[ 9  5 16  1 15]

 Iteracion no: 3
Mejor particula $1
fit:2750 x:[ 9  5 22  1 24]
Mejor particula $2
fit:2592.050909110547 x:[13.80077296  4.88058233  9.53070765  3.58672215 23.16849413]
Mejor particula $3
fit:2264 x:[ 1  5 21  2

In [15]:
p1 = [14, 3, 24, 0, 21]

problema.check(p1)

True